In [11]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import shelve
import random
import itertools
from pandas import Series,DataFrame,read_csv, merge,concat,read_hdf
from omero.gateway import BlitzGateway
from matplotlib.widgets import Slider
from matplotlib import gridspec
from sklearn.externals import joblib
from contextlib import closing


%matplotlib nbagg
plt.rcParams['image.cmap']='gray'

#hard coded dirs
dbpath='/home/ubuntu/idr_homes/szleo/features/idr0008-rohn-actinome/screenB/output/'
dataDir='/home/ubuntu/data'

In [3]:
def getImIDs(plns, wells,fields):
    lids=list()
    try:
        conn = BlitzGateway('demo', 'cambridge', host='orca-3.openmicroscopy.org', port=4064)
        connected = conn.connect()

        for pln,wstr,f in zip(plns,wells,fields):
        
            r=ord(wstr[0:1])-65
            c=int(wstr[1:])-1
            
#            print pln,wstr,f,r,c

            Q='''select well from Well as well 
            left outer join fetch well.wellSamples as ws 
            left outer join fetch ws.image as img 
            where well.plate.name = \'''' + pln+ '\' and well.row='+str(r)+' and well.column='+str(c)

            w=conn.getQueryService().findAllByQuery(Q,None)[0]
            im = w.getWellSample(int(f)-1).getImage()
            lids.append(im.getId().getValue())
    finally:
        conn._closeSession()
    
    return lids

In [4]:
def getMapAnnotationsAsDicts(imids):
    la=list()
    try:
        conn = BlitzGateway('demo', 'cambridge', host='orca-3.openmicroscopy.org', port=4064)
        connected = conn.connect()

        for imid in imids:
            im= conn.getObject("Image",imid)
            a=im.getAnnotation()
            la.append(dict(a.getValue()))
    finally:
        conn._closeSession()
    
    return la

In [5]:
def db2pd(fin):
#from https://gist.github.com/manics/bc003272b038027faf653d576f3c9393

    # f = "plate1_1_013_5_features.db"

#    print "Opening %s" % fin
    with closing(shelve.open(fin, "r")) as f:
        values = list(f.itervalues())

    x = values[0]

    header_meta = []
    header_ft = []
    header_ftn = []
    # width_ft = []

    for k in sorted(x.keys()):
        v = x[k]
        if isinstance(v, list):
            header_ft.append(k)
            header_ftn.extend('%s_%02d' % (k, i) for i in xrange(len(v)))
            # width_ftn.append(len(v))
        else:
            header_meta.append(k)

    assert len(header_ft) == 124
    assert len(header_meta) == 11

#    print "Processing %s" % fin
    rows = []
    for vs in values:
        meta = [vs[h] for h in header_meta]
        ft = [vs[h] for h in header_ft]
        r = itertools.chain.from_iterable([meta] + ft)
        rows.append(list(r))

#    print "Creating dataframe %s" % fin
    df = DataFrame(rows, columns=(header_meta + header_ftn))

    #print "Saving %s" % fout
    #df.to_hdf(fout, 'wndcharm', complevel=9, complib='zlib')

    return df


## preping data

In [6]:
#metadata: linking omero, features, annotations

s2w=read_csv(os.path.join(dataDir,'series_to_well.tsv'),sep='\t')
lids=getImIDs(s2w.PLATE, s2w.WELL,s2w.FIELD)
s2w['imIds']=lids
la=getMapAnnotationsAsDicts(s2w.imIds)
s2w['annotations']=la
s2w.to_csv(os.path.join(dataDir,'series_to_well_IDAnnotation.tsv'),sep='\t')

KeyboardInterrupt: 

In [7]:
s2w=read_csv(os.path.join(dataDir,'series_to_well_IDAnnotation.tsv'),sep='\t')

In [8]:
pln='Plate10_Actinome1'
df=DataFrame()
for s in range(s2w[s2w.PLATE==pln].SERIES.max()):
    if s%50==0:
        print s
    fin=os.path.join(dbpath, pln,'116_86',pln+'_'+str(s)+'_features'+'.db')
    df=concat((df,db2pd(fin)))

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100


In [12]:
fout=os.path.join(dataDir,pln+'_features'+'.h5')
df.to_hdf(fout, 'wndcharm', complevel=9, complib='zlib')

/home/ubuntu/anaconda2/lib/python2.7/site-packages/pandas/io/pytables.py:257: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->unicode,key->axis0] [items->None]

  f(store)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/pandas/io/pytables.py:257: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->unicode,key->block0_items] [items->None]

  f(store)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/pandas/io/pytables.py:257: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->unicode,key->block1_items] [items->None]

  f(store)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:939: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directl

## loading stuff

In [ ]:
s2w=read_csv(os.path.join(dataDir,'series_to_well_IDAnnotation.tsv'),sep='\t')
df=read_hdf() #TODO

In [13]:
pca= joblib.load(os.path.join(dataDir,'PCA_4condentationwells.pkl')) 

## DAPI PCA

In [19]:
dfdapi=df[df.c==0].iloc[:,11:]
dfdapi=scale(dfdapi)
datPCA=pca.transform(dat)

In [18]:
df.iloc[:,11:21].head()

,chebyshev_coefficients_00,chebyshev_coefficients_01,chebyshev_coefficients_02,chebyshev_coefficients_03,chebyshev_coefficients_04,chebyshev_coefficients_05,chebyshev_coefficients_06,chebyshev_coefficients_07,chebyshev_coefficients_08,chebyshev_coefficients_09
0,1,2,0,2,0,1,0,6,10,14
1,1,1,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,2
3,2,0,0,0,0,0,0,0,0,1
4,2,0,0,0,0,0,0,0,0,1
